In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
# Read in csv file containing raw data
data = pd.read_csv("resources/nuforc_reports.csv")
display(data.head(3))

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12T18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.343152,-77.408582
1,I think that I may caught a UFO on the NBC Nig...,Rocky Hill,CT,2019-03-22T18:30:00,circle,3-5 seconds,Occurred : 3/22/2019 18:30 (Entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29T00:00:00,41.664800,-72.639300
2,I woke up late in the afternoon 3:30-4pm. I we...,NaN,NaN,NaN,NaN,NaN,Occurred : 4/1/2019 15:45 (Entered as : April...,http://www.nuforc.org/webreports/145/S145556.html,I woke up late in the afternoon 3:30-4pm. I w...,NaN,NaN,NaN


In [3]:
# Check for null values
display(data.isnull().sum())

summary              30
city                234
state              5235
date_time          1187
shape              2498
duration           3171
stats                37
report_link           0
text                 55
posted             1187
city_latitude     16112
city_longitude    16112
dtype: int64

In [4]:
# Check number of states included and unique values in the state column
print("Total Number of States:")
print(data["state"].nunique())
print("States Included in Data:")
print(data["state"].unique())

Total Number of States:
65
States Included in Data:
['VA' 'CT' nan 'ON' 'NY' 'TX' 'AZ' 'IN' 'FL' 'NM' 'IA' 'SC' 'CA' 'CO' 'RI'
 'TN' 'PA' 'NJ' 'WA' 'MI' 'MO' 'IL' 'OH' 'OR' 'MA' 'WY' 'KY' 'NC' 'AR'
 'OK' 'QC' 'AB' 'AL' 'SD' 'MD' 'ME' 'GA' 'MN' 'NV' 'LA' 'WI' 'UT' 'MS'
 'BC' 'MT' 'NH' 'ID' 'NB' 'WV' 'DC' 'NE' 'KS' 'MB' 'AK' 'NS' 'SK' 'NT'
 'VT' 'ND' 'HI' 'DE' 'NL' 'PR' 'YT' 'PE' 'QB']


In [5]:
# Extra states are all Canadian states, no typos
# Drop extra states - since we're only looking at the USA
data.drop(data[data["state"] == "ON"].index, inplace=True)
data.drop(data[data["state"] == "QC"].index, inplace=True)
data.drop(data[data["state"] == "AB"].index, inplace=True)
data.drop(data[data["state"] == "BC"].index, inplace=True)
data.drop(data[data["state"] == "NB"].index, inplace=True)
data.drop(data[data["state"] == "MB"].index, inplace=True)
data.drop(data[data["state"] == "NS"].index, inplace=True)
data.drop(data[data["state"] == "SK"].index, inplace=True)
data.drop(data[data["state"] == "NT"].index, inplace=True)
data.drop(data[data["state"] == "NL"].index, inplace=True)
data.drop(data[data["state"] == "PR"].index, inplace=True)
data.drop(data[data["state"] == "YT"].index, inplace=True)
data.drop(data[data["state"] == "PE"].index, inplace=True)
data.drop(data[data["state"] == "QB"].index, inplace=True)

# Change DC to MD to count it as part of Maryland
data.loc[data["state"] == "DC", "state"] = "MD"

In [6]:
# Check unique values in the "shape" column
display(data["shape"].unique())

array(['light', 'circle', nan, 'cigar', 'disk', 'unknown', 'oval',
       'other', 'sphere', 'changing', 'formation', 'flash', 'chevron',
       'triangle', 'cylinder', 'fireball', 'diamond', 'egg', 'teardrop',
       'rectangle', 'cone', 'cross'], dtype=object)

In [7]:
# Remove all rows containing the "unknown" value in the shape column
data.drop(data[data["shape"] == "unknown"].index, inplace=True)

In [8]:
# Data = data.dropna(axis=0, subset=["state"], inplace=True)
data = data[data["state"].notna()]

In [9]:
# Drop all unneeded columns from our dataframe
data = data.drop(["summary", "duration", "stats", "report_link", "text", "posted", "city_latitude", "city_longitude"], axis=1)
display(data.head())

,city,state,date_time,shape
0,Chester,VA,2019-12-12T18:43:00,light
1,Rocky Hill,CT,2019-03-22T18:30:00,circle
4,Peoria,NY,2009-03-15T18:00:00,cigar
5,Kirbyville,TX,2019-04-02T20:25:00,disk
7,Gold Canyon,AZ,2019-04-10T17:00:00,circle


In [10]:
# Check if any null values are left
display(data.isnull().sum())

city          110
state           0
date_time     943
shape        2130
dtype: int64

In [11]:
# Drop all remaining null values
data = data.dropna()

In [12]:
# Convert original timestamp into usable timestamp
date_format = "%Y-%m-%dT%H:%M:%S"
date_time = []

# Loop conversion through each row
for x in range(len(data["date_time"])):
    ts = time.strptime(data.iloc[x,2], date_format)
    dt = time.strftime("%Y-%m-%d %H:%M:%S", ts)
    date_time.append(dt)

In [13]:
# Change new timestamp to correct datetime object type
data["timestamp"] = date_time
data["converted_timestamp"] = pd.to_datetime(data["timestamp"], format="%Y-%m-%d %H:%M:%S")
data.head()

,city,state,date_time,shape,timestamp,converted_timestamp
0,Chester,VA,2019-12-12T18:43:00,light,2019-12-12 18:43:00,2019-12-12 18:43:00
1,Rocky Hill,CT,2019-03-22T18:30:00,circle,2019-03-22 18:30:00,2019-03-22 18:30:00
4,Peoria,NY,2009-03-15T18:00:00,cigar,2009-03-15 18:00:00,2009-03-15 18:00:00
5,Kirbyville,TX,2019-04-02T20:25:00,disk,2019-04-02 20:25:00,2019-04-02 20:25:00
7,Gold Canyon,AZ,2019-04-10T17:00:00,circle,2019-04-10 17:00:00,2019-04-10 17:00:00


In [14]:
# Create columns for each individual piece of the timestamp
data["year"] = data["converted_timestamp"].dt.year
data["month"] = data["converted_timestamp"].dt.month
data["day"] = data["converted_timestamp"].dt.day
data["hour"] = data["converted_timestamp"].dt.hour
data["minute"] = data["converted_timestamp"].dt.minute
data = data[["city", "state", "year", "month", "day", "hour", "minute", "converted_timestamp", "shape"]]
data.head()

,city,state,year,month,day,hour,minute,converted_timestamp,shape
0,Chester,VA,2019,12,12,18,43,2019-12-12 18:43:00,light
1,Rocky Hill,CT,2019,3,22,18,30,2019-03-22 18:30:00,circle
4,Peoria,NY,2009,3,15,18,0,2009-03-15 18:00:00,cigar
5,Kirbyville,TX,2019,4,2,20,25,2019-04-02 20:25:00,disk
7,Gold Canyon,AZ,2019,4,10,17,0,2019-04-10 17:00:00,circle


In [15]:
# Save the clean dataframe to a new csv
data.to_csv("output/clean_data.csv", index=False)